In [ ]:
!pip install seaborn matplotlib scikit-learn

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('float_format', '{:f}'.format)
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [ ]:
print(os.environ['TNS_ADMIN'])

In [ ]:
df = pd.read_json('/Users/mattgunnin/Sites/AI/20_GPT/gpt_lolcoach/data/lol/export.json')

df.head(5)

In [ ]:
df['champ1'].unique() # all champion names

In [ ]:
champ_list = df['champ1'].unique().tolist()

print(type(champ_list))

In [ ]:
print(champ_list)

In [ ]:
# Data Exploration
df.describe().transpose()

In [ ]:
print(df.columns)

In [ ]:
df.info()

In [ ]:
# Cleaning data
df.isna().sum()

In [ ]:
df = df.dropna() # we drop these null values and corresponding rows.

In [ ]:
sns.countplot(df['champ1'], label="Count")
plt.show()

In [ ]:
df.head(5)

In [ ]:
# Drop match_id column
df = df.drop(['match_id'], axis=1)

In [ ]:
df.head(5)

In [ ]:
train_dataset = df.sample(frac=0.8, random_state=0) # 80-20 train-test splitting.
test_dataset = df.drop(train_dataset.index) # drop all rows present in the train dataset.

In [ ]:
# Let's look at the shape of our dataset after removing and modifying column values.
df.head(5)

In [ ]:
# Splitting features (what we use to predict) from labels (what we want to predict)
# We want to predict the 'win' variable.
# The rest of variables will be inputs.

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('win') # returns column 'win'
test_labels = test_features.pop('win') # returns column 'win'

In [ ]:
le = LabelEncoder()

le = le.fit(champ_list) # fit the label encoder with the whole champion list.
#le.classes_

train_features = train_features.apply(lambda x: le.transform(x))
test_features = test_features.apply(lambda x: le.transform(x))

In [ ]:
# Normalization
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [ ]:
logreg = LogisticRegression()
logreg.fit(train_features, train_labels)
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(train_features, train_labels)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(logreg.score(test_features, test_labels)))

In [ ]:
logreg.predict(test_features)

In [ ]:
new_data = {
    'champ1': ['Xayah', 'Karma', 'Xerath', 'Gragas', 'Chogath'],
    'champ2': ['Tristana', 'Lulu', 'Syndra', 'Sejuani', 'Gnar']
}
new_df = pd.DataFrame(new_data)

In [ ]:
#le.transform(new_df['champ1'])

new_df = new_df.apply(lambda x: le.transform(x))

In [ ]:
new_df.tail(5)

In [ ]:
result = logreg.predict(new_df)
print(result)

In [ ]:
inverse_prediction = new_df.apply(lambda x: le.inverse_transform(x))

In [ ]:
def find_winner(lst):
    return max(set(lst), key=lst.count)

In [ ]:
winner_prediction = find_winner(result.tolist())

In [ ]:
if winner_prediction == 1:
    print('Predicted winner is team 1: \n{}'.format(str(inverse_prediction['champ1'])))
else:
     print('Predicted winner is team 2: \n{}'.format(str(inverse_prediction['champ2'])))
